#MVP Engenharia de Dados - 2025
Thomas A. S. Abrantes


### Setup para rodar o notebook com os dados
- Imports das bibliotecas para consumo dos dados.

In [0]:
#bibioteca para uso do comando com Pandas
import pandas as pd

#biblioteca para carga de dados e request dos dados
import io
import requests

#Criando Banco de Dados (schema) com os Relatórios (Visões)


In [0]:
%sql
-- Removendo Schema para recriar
DROP SCHEMA IF EXISTS IMDB_RELATORIO CASCADE;

-- CRIANDO O BANCO DE DADOS (schema)
CREATE SCHEMA IMDB_RELATORIO;

#Relatórios

Neste notebook podemos consultar a criação das visões (consultas) utilizando como fonte a base (IMDB_DB_GOLD) **Camada GOLD** de forma a responder a cada uma das perguntas.

###Quantidade de títulos lançados no Brasil por categoria e década (80, 90, etc.).

In [0]:
%sql
CREATE OR REPLACE VIEW IMDB_RELATORIO.VW_LANCAMENTO_POR_TIPO_DECADA
AS
  SELECT
    CASE WHEN DECADA > 0 THEN concat('19', DECADA,'0') ELSE concat('20', DECADA,'0') END DECADA,
    CATEGORIA,
    QUANTIDADE TITULOS_LANCADOS
  FROM (
    SELECT 
      CATEGORIA, COUNT(0) QUANTIDADE, substring(ANO_LANCAMENTO, 2, 1) DECADA
    FROM imdb_db_gold.tb_filme_lancamento_brasil
    GROUP BY CATEGORIA, substring(ANO_LANCAMENTO, 2, 1)
  ) CONSULTA
  ORDER BY decada, TITULOS_LANCADOS DESC, CATEGORIA
  ;

  -- EXIBINDO RESULTADO
SELECT * FROM IMDB_RELATORIO.VW_LANCAMENTO_POR_TIPO_DECADA;

DECADA,CATEGORIA,TITULOS_LANCADOS
1980,short,76
1990,movie,41553
1990,short,5383
1990,video,1127
2000,movie,31863
2000,short,12476
2000,video,4212


Databricks visualization. Run in Databricks to view.

###Qual categoria teve mais lançamentos no Brasil por década?

In [0]:
%sql
CREATE OR REPLACE VIEW IMDB_RELATORIO.VW_CATEGORIAS_COM_MAIS_LANCAMENTO_BRASIL
AS
SELECT
  VISAO_GERAL.DECADA,
  CASE VISAO_GERAL.CATEGORIA
    WHEN 'movie' THEN 'Filme'
    WHEN 'video' THEN 'Vídeo (dvd, vhs, etc.)'
    WHEN 'short' THEN 'Curta metragem'
  ELSE VISAO_GERAL.CATEGORIA end CATEGORIA_MAIOR_QUANTIDADE_LANCAMENTOS,
  LANCAMENTOS.TITULOS_LANCADOS QUANTIDADE
FROM 
  IMDB_RELATORIO.VW_LANCAMENTO_POR_TIPO_DECADA VISAO_GERAL,
  (
    SELECT DECADA, MAX(TITULOS_LANCADOS) TITULOS_LANCADOS 
    FROM IMDB_RELATORIO.VW_LANCAMENTO_POR_TIPO_DECADA
    GROUP BY DECADA
  ) LANCAMENTOS
WHERE VISAO_GERAL.DECADA = LANCAMENTOS.DECADA AND VISAO_GERAL.TITULOS_LANCADOS = LANCAMENTOS.TITULOS_LANCADOS
ORDER BY VISAO_GERAL.DECADA
;

-- EXIBINDO RESULTADO
SELECT * FROM IMDB_RELATORIO.VW_CATEGORIAS_COM_MAIS_LANCAMENTO_BRASIL;

DECADA,CATEGORIA_MAIOR_QUANTIDADE_LANCAMENTOS,QUANTIDADE
1980,Curta metragem,76
1990,Filme,41553
2000,Filme,31863


Databricks visualization. Run in Databricks to view.

###Os últimos 10 títulos que possuem nome exclusivo no Brasil (Com nome diferente do nome original ou comercial).

- **Conforme análise** na criação da tabela **IMDB_DB_BRONZE.tb_title_akas**, identificamos que não há título com campo isOriginalTitle  = 1 (True) com o campo region preenchido. 
- Mesmo que o titulo TITULO_ORIGINAL seja igual o TITULO de lançamento e descrito em português, não podemos afirmar que seja um Título produzido no Brasil, devido a falta da informação de ser um título Original. 
- Desta forma todos os titulos duplicados que tenha a coluna IDIOMA com valor "pt" e outra com valor nulo (NULL) e analisando as informações fornecidas pelo Dicionário de Dados; concluo que o registro do título com campo IDIOMA preenchido é um título legendado ou dublado.

In [0]:
%sql
CREATE OR REPLACE VIEW IMDB_RELATORIO.VW_TITULOS_COM_NOME_EXCLUSIVO_NO_BRASIL
AS
SELECT 
  TITULO TITULO_BRASIL,
  TITULO_ORIGINAL TITULO_ORIGINAL,
  TITULO_PRIMARIO TITULO_COMERCIAL,
  ANO_LANCAMENTO
FROM 
  IMDB_DB_GOLD.tb_filme_lancamento_brasil
WHERE
  TRIM(TITULO) <> TRIM(TITULO_ORIGINAL) OR TRIM(TITULO) <> TRIM(TITULO_PRIMARIO)
GROUP BY
  ANO_LANCAMENTO,
  TITULO_PRIMARIO, 
  TITULO_ORIGINAL, 
  TITULO
HAVING ANO_LANCAMENTO <= 2025
ORDER BY ANO_LANCAMENTO DESC, TITULO
LIMIT 10
;

-- EXIBINDO RESULTADO 20 PRIMEIROS REGISTROS
SELECT * FROM IMDB_RELATORIO.VW_TITULOS_COM_NOME_EXCLUSIVO_NO_BRASIL LIMIT 20;

TITULO_BRASIL,TITULO_ORIGINAL,TITULO_COMERCIAL,ANO_LANCAMENTO
366,UN AÑO Y UN DÍA,UN AÑO Y UN DÍA,2025
A ARMADILHA DO COELHO,RABBIT TRAP,RABBIT TRAP,2025
A BATALHA DO MAR CHINÊS,JIAO LONG XING DONG,OPERATION HADAL,2025
A CASA DO MICKEY: PARQUE DA MORTE,MICKEY'S SLAYHOUSE,MICKEY'S SLAYHOUSE,2025
A EMPREGADA,THE HOUSEMAID,THE HOUSEMAID,2025
A FAZENDA DOS ANIMAIS,ANIMAL FARM,ANIMAL FARM,2025
A FÚRIA DO LEÃO,LION FIST,LION FIST,2025
A GAROTA MAIS LINDA DO MUNDO,THE MOST BEAUTIFUL GIRL IN THE WORLD,THE MOST BEAUTIFUL GIRL IN THE WORLD,2025
A GUERRA DAS PALAVRAS,WORDS OF WAR,WORDS OF WAR,2025
A LISTA DO PERIGO: QUEM VAI ESCAPAR?,THE HIT-LIST,THE HIT-LIST,2025


###Qual a porcentagem de pessoas que atuaram como Ator ou Atriz em Títulos do tipo Filmes, Curtas ou Vídeo, lançados no Brasil?

- Pela profissão da pessoa será definido seu sexo como Masculino (actor) ou Feminino (actress)

In [0]:
%sql
CREATE OR REPLACE VIEW IMDB_RELATORIO.VW_PORCENTAGEM_SEXO_EM_TITULOS_NO_BRASIL
AS
SELECT 
  TOTAL.TODOS,
  SEXO_FEMININO.TOTAL FEMININO,
  SEXO_MASCULINO.TOTAL MASCULINO,
  decimal(SEXO_FEMININO.TOTAL * 100 / TOTAL.TODOS) PORCENTAGEM_FEMININO,
  decimal(SEXO_MASCULINO.TOTAL * 100 / TOTAL.TODOS) PORCENTAGEM_MASCULINO
FROM
  (SELECT COUNT(0) AS TODOS
    FROM IMDB_DB_GOLD.TB_ATOR_TITULO 
    WHERE PROFISSAO1 = 'actor' OR PROFISSAO1 = 'actress' OR PROFISSAO2 = 'actor' OR PROFISSAO2 = 'actress' OR PROFISSAO3 = 'actor' OR PROFISSAO3 ='actress') TOTAL,
  (SELECT COUNT(0)  AS TOTAL FROM IMDB_DB_GOLD.TB_ATOR_TITULO WHERE PROFISSAO1 = 'actress' OR PROFISSAO2 = 'actress' OR PROFISSAO3 = 'actress') SEXO_FEMININO,
  (SELECT COUNT(0) AS TOTAL FROM IMDB_DB_GOLD.TB_ATOR_TITULO WHERE PROFISSAO1 = 'actor' OR PROFISSAO2 = 'actor' OR PROFISSAO3 = 'actor') SEXO_MASCULINO
;

SELECT * FROM IMDB_RELATORIO.VW_PORCENTAGEM_SEXO_EM_TITULOS_NO_BRASIL;

TODOS,FEMININO,MASCULINO,PORCENTAGEM_FEMININO,PORCENTAGEM_MASCULINO
895200,276415,618854,31,69


Databricks visualization. Run in Databricks to view.

##Porcentagem de atores do sexo feminino e masculino que atuaram nos títulos do tipo Filme em cada década existente na base.
- Para o resultado, criação da View IMDB_RELATORIO.VW_TOTAL_POR_DECADA_GENERO_ATOR que vai estruturar a consulta para o cálculo das porcentagens.
- Obter a porcentagem de atores do sexo feminino e masculino que atuaram nos títulos do tipo Filme em cada década existente na base.

In [0]:
%sql
-- Criando visão geral para o relatório
CREATE OR REPLACE VIEW IMDB_RELATORIO.VW_TOTAL_POR_DECADA_SEXO_ATOR
AS
SELECT 
  SEXO,
  DECADA,
  TOTAL
FROM
  (
    SELECT
      CASE 
        WHEN ATOR_TITULO.SEXO = 'M' THEN 'MASCULINO'
        WHEN ATOR_TITULO.SEXO = 'F' THEN 'FEMININO'
      ELSE 'NÃO INFORMADO'
      END SEXO,
      CASE WHEN substring(FILMES.ANO_LANCAMENTO, 2, 1) > 0 
        THEN concat('19', substring(FILMES.ANO_LANCAMENTO, 2, 1),'0') 
        ELSE concat('20', substring(FILMES.ANO_LANCAMENTO, 2, 1),'0') 
      END DECADA,
      count(0) TOTAL
    FROM
      IMDB_DB_GOLD.TB_ATOR_TITULO_BRASIL AS ATOR_TITULO,
      IMDB_DB_GOLD.TB_FILME_LANCAMENTO_BRASIL FILMES
    WHERE FILMES.CHAVE_TITULO = ATOR_TITULO.CHAVE_TITULO
    GROUP BY 
      ATOR_TITULO.SEXO,  
      substring(FILMES.ANO_LANCAMENTO, 2, 1)
    ORDER BY DECADA
  ) AS CONSULTA;

  SELECT * FROM IMDB_RELATORIO.VW_TOTAL_POR_DECADA_SEXO_ATOR;

SEXO,DECADA,TOTAL
NÃO INFORMADO,1980,3
FEMININO,1980,26
MASCULINO,1980,70
NÃO INFORMADO,1990,1692463
FEMININO,1990,859893
MASCULINO,1990,2560843
NÃO INFORMADO,2000,1198201
FEMININO,2000,448839
MASCULINO,2000,937719


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Criando consulta
SELECT 
  TODOS.TOTAL,
  decimal(FEMININO.TOTAL*100 / TODOS.TOTAL) PORCENTAGEM_FEMININO,
  decimal(MASCULINO.TOTAL*100 / TODOS.TOTAL) PORCENTAGEM_MASCULINO,
  decimal(NAO_INFORMADO.TOTAL*100 / TODOS.TOTAL) PORCENTAGEM_NAO_INFORMADO,
  TODOS.DECADA
FROM
  (SELECT SUM(TOTAL) TOTAL, DECADA FROM IMDB_RELATORIO.VW_TOTAL_POR_DECADA_SEXO_ATOR GROUP BY DECADA) AS TODOS,
  (SELECT TOTAL, DECADA FROM IMDB_RELATORIO.VW_TOTAL_POR_DECADA_SEXO_ATOR WHERE SEXO = 'MASCULINO') AS MASCULINO,
  (SELECT TOTAL, DECADA FROM IMDB_RELATORIO.VW_TOTAL_POR_DECADA_SEXO_ATOR WHERE SEXO = 'FEMININO') AS FEMININO,
  (SELECT TOTAL, DECADA FROM IMDB_RELATORIO.VW_TOTAL_POR_DECADA_SEXO_ATOR WHERE SEXO = 'NÃO INFORMADO') AS NAO_INFORMADO
WHERE 
  TODOS.DECADA = FEMININO.DECADA
  AND FEMININO.DECADA = MASCULINO.DECADA
  AND MASCULINO.DECADA = NAO_INFORMADO.DECADA
ORDER BY TODOS.DECADA
;

TOTAL,PORCENTAGEM_FEMININO,PORCENTAGEM_MASCULINO,PORCENTAGEM_NAO_INFORMADO,DECADA
99,26,71,3,1980
5113199,17,50,33,1990
2584759,17,36,46,2000


###Os primeiros 20 títulos lançados no Brasil, onde o Diretor(a) atuou como Ator/Atriz.

In [0]:
%sql
SELECT 
  ATOR_TITULO.NOME_ARTISTICO,
  FILME.TITULO_ORIGINAL,
  FILME.ANO_LANCAMENTO,
  FILME.REGIAO,
  ATOR_TITULO.PROFISSAO1,
  ATOR_TITULO.PROFISSAO2,
  ATOR_TITULO.PROFISSAO3
FROM 
  IMDB_DB_GOLD.TB_ATOR_TITULO AS ATOR_TITULO,
  IMDB_DB_GOLD.TB_FILME_LANCAMENTO_BRASIL AS FILME
WHERE 
  ATOR_TITULO.DIRETOR_TITULO = 'S'
  AND (ATOR_TITULO.PROFISSAO1 LIKE 'act%' OR ATOR_TITULO.PROFISSAO2 LIKE 'act%' OR ATOR_TITULO.PROFISSAO3 LIKE 'act%')
  AND (
    FILME.CHAVE_TITULO = ATOR_TITULO.TCONST1 
    OR FILME.CHAVE_TITULO = ATOR_TITULO.TCONST2 
    OR FILME.CHAVE_TITULO = ATOR_TITULO.TCONST3 
    OR FILME.CHAVE_TITULO = ATOR_TITULO.TCONST4
  )  
GROUP BY 
  ATOR_TITULO.NOME_ARTISTICO,
  FILME.TITULO_ORIGINAL,
  FILME.ANO_LANCAMENTO,
  FILME.REGIAO,
  ATOR_TITULO.PROFISSAO1,
  ATOR_TITULO.PROFISSAO2,
  ATOR_TITULO.PROFISSAO3
ORDER BY FILME.ANO_LANCAMENTO, ATOR_TITULO.NOME_ARTISTICO, FILME.TITULO_ORIGINAL
LIMIT 20
;


NOME_ARTISTICO,TITULO_ORIGINAL,ANO_LANCAMENTO,REGIAO,PROFISSAO1,PROFISSAO2,PROFISSAO3
Georges Méliès,LE MANOIR DU DIABLE,1896,BR,director,actor,producer
Georges Méliès,L'ÉCLIPSE DU SOLEIL EN PLEINE LUNE,1907,BR,director,actor,producer
Benjamin Oliveira,OS GUARANIS,1908,BR,actor,director,writer
Francisco Marzullo,OS ESTRANGULADORES,1908,BR,actor,director,cinematographer
Francisco Marzullo,"ROUBO DOS 1,400 CENTOS",1908,BR,actor,director,cinematographer
Antônio Serra,LUCRÉCIA BORGIA,1909,BR,director,actor,null
Antônio Serra,PEGA NA CHALEIRA,1909,BR,director,actor,null
Antônio Serra,UM CAVALHEIRO DEVERAS OBSEQUIOSO,1909,BR,director,actor,null
Antônio Serra,UMA LICAO DE MAXIXE,1909,BR,director,actor,null
Eduardo Leite,A VIÚVA ALEGRE,1909,BR,actor,director,writer


##Ranking dos diretores com mais de 7 títulos lançados no Brasil.
- Diretores que teve participação na direção em mais de 7 de títulos lançados no Brasil.

In [0]:
%sql
-- PARA NAO DEIXAR O NOTEBOOK GRANDE NA LEITURA NO GIT EXIBINDO OS 20 PRIMEIROS RESULTADOS
SELECT
  NOME_ARTISTICO,
  count(CHAVE_TITULO) TOTAL
FROM 
(
  SELECT
    ATOR_TITULO.NOME_ARTISTICO,
    ATOR_TITULO.CHAVE_TITULO
  FROM 
    IMDB_DB_GOLD.TB_ATOR_TITULO_BRASIL AS ATOR_TITULO,
    IMDB_DB_GOLD.TB_FILME_LANCAMENTO_BRASIL AS FILME
  WHERE 
    ATOR_TITULO.DIRETOR_TITULO = 'S'
    AND TRIM(FILME.CHAVE_TITULO) = TRIM(ATOR_TITULO.CHAVE_TITULO)
  GROUP BY ATOR_TITULO.NOME_ARTISTICO,
    ATOR_TITULO.CHAVE_TITULO
) AS DIRETOR_TITULOS
GROUP BY NOME_ARTISTICO
HAVING count(CHAVE_TITULO) >= 7
ORDER BY TOTAL DESC
;

NOME_ARTISTICO,TOTAL
Pedro Murad,9
John Waters,8
Luciano Mello,8
Bruno de Oliveira,8
Bruno Costa,8
Marcelo Leme,8
Leonardo Martinelli,8
Anna Azevedo,7
George Miller,7
Joaquim Haickel,7
